In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import mlflow
import mlflow.pytorch
import optuna
import pickle, joblib


#### Verificação de valores e criação de features

In [45]:
df_acao_bruto = pd.read_csv('base_historica\\AAPL_7anos.csv')
df_acao_bruto['Date'] = pd.to_datetime(df_acao_bruto['Date'])
df_acao_bruto.info()

df_acao = df_acao_bruto[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
df_acao['Weekday'] = df_acao_bruto['Date'].dt.weekday
df_acao['Month'] = df_acao_bruto['Date'].dt.month
df_acao['Year'] = df_acao_bruto['Date'].dt.year
df_acao['day_sin'] = np.sin(2 * np.pi * df_acao_bruto['Date'].dt.dayofyear / 365)
df_acao['day_cos'] = np.cos(2 * np.pi * df_acao_bruto['Date'].dt.dayofyear / 365)

df_acao.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1968 non-null   datetime64[ns]
 1   Open       1968 non-null   float64       
 2   High       1968 non-null   float64       
 3   Low        1968 non-null   float64       
 4   Close      1968 non-null   float64       
 5   Adj Close  1968 non-null   float64       
 6   Volume     1968 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 107.8 KB


,Date,Open,High,Low,Close,Volume,Weekday,Month,Year,day_sin,day_cos
0,2017-01-03,28.950001,29.082500,28.690001,29.037500,115127600,1,1,2017,0.051620,0.998667
1,2017-01-04,28.962500,29.127501,28.937500,29.004999,84472400,2,1,2017,0.068802,0.997630
2,2017-01-05,28.980000,29.215000,28.952499,29.152500,88774400,3,1,2017,0.085965,0.996298
3,2017-01-06,29.195000,29.540001,29.117500,29.477501,127007600,4,1,2017,0.103102,0.994671
4,2017-01-09,29.487499,29.857500,29.485001,29.747499,134247600,0,1,2017,0.154309,0.988023


#### Normalização dos dados 

In [46]:
cols_norm = ['Open', 'High', 'Low', 'Close', 'Volume', 'Weekday', 'Month', 'Year']

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_acao[cols_norm])
df_acao[cols_norm] = scaler.transform(df_acao[cols_norm])

df_acao.head()


,Date,Open,High,Low,Close,Volume,Weekday,Month,Year,day_sin,day_cos
0,2017-01-03,-1.000000,-1.000000,-1.000000,-0.999687,-0.570271,-0.5,-1.0,-1.0,0.051620,0.998667
1,2017-01-04,-0.999880,-0.999568,-0.997594,-1.000000,-0.714908,0.0,-1.0,-1.0,0.068802,0.997630
2,2017-01-05,-0.999711,-0.998728,-0.997448,-0.998578,-0.694610,0.5,-1.0,-1.0,0.085965,0.996298
3,2017-01-06,-0.997639,-0.995610,-0.995845,-0.995445,-0.514219,1.0,-1.0,-1.0,0.103102,0.994671
4,2017-01-09,-0.994820,-0.992563,-0.992273,-0.992843,-0.480059,-1.0,-1.0,-1.0,0.154309,0.988023


#### Salvar arquivo tratado

In [31]:
df_acao.to_csv(f"AAPL_7_years_data_norm.csv", index=False)

# Modelo 

In [32]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm1(x, (h0, c0))
        out = self.fc1(out[:, -1, :])
        out, _ = self.lstm2(x, (h0, c0))
        out = self.fc2(out[:, -1, :])
        return out



def evaluate_model2(model, criterion):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    average_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {average_test_loss:.4f}")
    mlflow.log_metric("test_loss", average_test_loss)
    


def train_model():
    model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    mlflow.set_experiment("Predicao_LSTM")
    with mlflow.start_run():
        mlflow.log_params({
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "sequence_length": sequence_length,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
        })

        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            
            for i, (sequences, labels) in enumerate(train_loader):
                sequences, labels = sequences.to(device), labels.to(device)

                # Forward pass
                outputs = model(sequences)
                loss = criterion(outputs, labels)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                
                # Log metrics every 100 batches
                if i % 100 == 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
                    mlflow.log_metric("train_loss", running_loss / (i+1), step=epoch * len(train_loader) + i)

        # Save the model
        example_input = torch.randn(1, sequence_length, input_size).to(device)
        example_input_np = example_input.cpu().numpy()
        mlflow.pytorch.log_model(model, "predicao_lstm", input_example=example_input_np)
        # evitar warning
    
    return model



def tester(trial):
    # range de parâmetros
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_loguniform("learning_rate", 0.0001, 0.01)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    num_epochs = trial.suggest_int("num_epochs", 30, 70)

    train_dataset = TensorDataset(train_X, train_y)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(test_X, test_y)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

    model = LSTM(input_size=data.shape[1], hidden_size=hidden_size, num_layers=num_layers, output_size=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        for sequences, labels in train_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    return val_loss / len(val_loader)


#### Estudo dos melhores parâmetros

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = df_acao[['Open', 'High', 'Low', 'Volume', 'Weekday', 'Month', 'Year', 'day_sin', 'day_cos']].values
targets = df_acao[['Close']].values

sequence_length = 20
pre_X, pre_y = [], []
for i in range(len(data) - sequence_length):
    pre_X.append(data[i:i+sequence_length])
    pre_y.append(targets[i+sequence_length])

X = torch.tensor(np.array(pre_X), dtype=torch.float32)
y = torch.tensor(np.array(pre_y), dtype=torch.float32)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=False)

estudo = optuna.create_study()
estudo.optimize(tester, n_trials=50)

print("Melhores parâmetros:", estudo.best_params)
print("Melhor loss:", estudo.best_value)


[I 2024-12-02 02:22:35,808] A new study created in memory with name: no-name-ae316e70-cdd9-4778-b0b5-9ead4d00c0dc
C:\Users\vitor\AppData\Local\Temp\ipykernel_1576\3639264506.py:94: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.0001, 0.01)
[I 2024-12-02 02:24:04,552] Trial 0 finished with value: 0.02530330582521856 and parameters: {'hidden_size': 128, 'num_layers': 1, 'learning_rate': 0.00012546524058606881, 'batch_size': 128, 'num_epochs': 67}. Best is trial 0 with value: 0.02530330582521856.
[I 2024-12-02 02:26:51,136] Trial 1 finished with value: 0.006129632138514093 and parameters: {'hidden_size': 256, 'num_layers': 1, 'learning_rate': 0.00018640690596259522, 'batch_size': 64, 'num_epochs': 46}. Best is trial 1 with value: 0.006129632138514093.
[I 2024-12-02 02:27:0

Melhores parâmetros: {'hidden_size': 64, 'num_layers': 1, 'learning_rate': 0.00864523171136962, 'batch_size': 128, 'num_epochs': 46}
Melhor loss: 0.001975661492906511


#### Treinamento do modelo com os melhores parâmetros

In [34]:
#best_params = {'hidden_size': 128, 'num_layers': 2, 'learning_rate': 0.005121810661211535, 'batch_size': 64, 'num_epochs': 30}
#best_params = {'hidden_size': 64, 'num_layers': 1, 'learning_rate': 0.00864523171136962, 'batch_size': 128, 'num_epochs': 46}

input_size = data.shape[1]
hidden_size = estudo.best_params['hidden_size']
num_layers = estudo.best_params['num_layers']
num_epochs = estudo.best_params['num_epochs']
batch_size = estudo.best_params['batch_size']
learning_rate = estudo.best_params['learning_rate']
sequence_length = 20
output_size = 1

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = train_model()

Epoch [1/46], Step [1/13], Loss: 0.5155
Epoch [2/46], Step [1/13], Loss: 0.0112
Epoch [3/46], Step [1/13], Loss: 0.0035
Epoch [4/46], Step [1/13], Loss: 0.0026
Epoch [5/46], Step [1/13], Loss: 0.0014
Epoch [6/46], Step [1/13], Loss: 0.0013
Epoch [7/46], Step [1/13], Loss: 0.0013
Epoch [8/46], Step [1/13], Loss: 0.0012
Epoch [9/46], Step [1/13], Loss: 0.0008
Epoch [10/46], Step [1/13], Loss: 0.0007
Epoch [11/46], Step [1/13], Loss: 0.0008
Epoch [12/46], Step [1/13], Loss: 0.0006
Epoch [13/46], Step [1/13], Loss: 0.0008
Epoch [14/46], Step [1/13], Loss: 0.0008
Epoch [15/46], Step [1/13], Loss: 0.0010
Epoch [16/46], Step [1/13], Loss: 0.0008
Epoch [17/46], Step [1/13], Loss: 0.0007
Epoch [18/46], Step [1/13], Loss: 0.0006
Epoch [19/46], Step [1/13], Loss: 0.0005
Epoch [20/46], Step [1/13], Loss: 0.0006
Epoch [21/46], Step [1/13], Loss: 0.0007
Epoch [22/46], Step [1/13], Loss: 0.0005
Epoch [23/46], Step [1/13], Loss: 0.0007
Epoch [24/46], Step [1/13], Loss: 0.0006
Epoch [25/46], Step [1/13

In [35]:
evaluate_model2(model, nn.MSELoss())

Test Loss: 0.0034


In [51]:
mlflow.end_run()
torch.save({
        'model_state_dict': model.state_dict(),
        'hyperparameters': {
            'input_size': input_size,
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'num_epochs': num_epochs,
            'batch_size': batch_size,
            'output_size': output_size,
            'learning_rate': learning_rate
        }
    }, 'modelo_aapl_lstm.pth')
joblib.dump(scaler, 'scaler.pkl')
    

['scaler.pkl']